In [ ]:

"""
dataset
--------------------------------------------------------------------------------------------------------------------------------
|                                                                                                                              |
--------------------------------------------------------------------------------------------------------------------------------

normal splitting
--------------------------------------------------------------------------------------------------------------------------------
|                              Train                                                   |      Eval          |     Test         |
--------------------------------------------------------------------------------------------------------------------------------

cross validation
--------------------------------------------------------------------------------------------------------------------------------
|                         |                       |                        |                      |                            |
--------------------------------------------------------------------------------------------------------------------------------



"""